In [ ]:
### '2016-01-04' ~ '2020-01-31'

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install pytorch-lightning  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 585 kB 6.8 MB/s 
     |████████████████████████████████| 596 kB 60.8 MB/s 
     |████████████████████████████████| 419 kB 66.0 MB/s 
     |████████████████████████████████| 140 kB 71.7 MB/s 
     |████████████████████████████████| 1.1 MB 45.0 MB/s 
     |████████████████████████████████| 144 kB 69.8 MB/s 
     |████████████████████████████████| 271 kB 74.8 MB/s 
     |████████████████████████████████| 94 kB 1.1 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
pip install pytorch-forecasting

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 127 kB 10.9 MB/s 
     |████████████████████████████████| 308 kB 45.1 MB/s 
     |████████████████████████████████| 81 kB 10.8 MB/s 
     |████████████████████████████████| 209 kB 71.3 MB/s 
     |████████████████████████████████| 78 kB 7.7 MB/s 
     |████████████████████████████████| 49 kB 6.6 MB/s 
     |████████████████████████████████| 146 kB 71.8 MB/s 
     |████████████████████████████████| 112 kB 74.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=8ede5e0cd0262581b57582a4eae14abbdd8a2281ab7a182e85be6399e593ad55
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [4]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from pytorch_forecasting import Baseline
from pytorch_forecasting.metrics import SMAPE

import joblib
import pickle
import optuna

warnings.filterwarnings('ignore')
%matplotlib inline

/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (SMAPE). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
                `from torchmetrics.utilities import check_forward_no_full_state`
                that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                default for now) or if `full_state_update=False` can be used safely.
                
  warnings.warn(*args, **kwargs)


### 함수정의

In [6]:
def sliding_windows(data, lookback_length, forecast_length):

    x = []
    y = []
    
    for i in range(lookback_length, len(data) - forecast_length + 1):
        _x = data[(i-lookback_length) : i]
        _y = data[i : (i + forecast_length)]
        x.append(_x)
        y.append(_y)
    return np.array(x), np.array(y)


def get_data_loader(X, y, batch_size):

    x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

    train_ds = TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train))
    train_dl = DataLoader(train_ds, batch_size = batch_size)

    val_ds = TensorDataset(torch.Tensor(x_val), torch.Tensor(y_val))
    val_dl = DataLoader(val_ds, batch_size = batch_size)

    input_size = x_train.shape[-1]

    return train_dl, val_dl, input_size

### Data loading && Preproces

In [7]:
scaler = MinMaxScaler()

data=pd.read_excel('/content/drive/MyDrive/교육/AI실무인증과정/학회 발표/data/data_full.xlsx')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1858 entries, 0 to 1857
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             1858 non-null   datetime64[ns]
 1   Account DOW      1858 non-null   object        
 2   REV OBD          1858 non-null   float64       
 3   OBD NET+FSC_KRW  1858 non-null   float64       
 4   OBD A/R_KRW      1858 non-null   float64       
 5   REV CPN          1858 non-null   float64       
 6   CPN NET+FSC_KRW  1858 non-null   float64       
 7   CPN A/R_KRW      1858 non-null   float64       
 8   REV TKT          1858 non-null   float64       
 9   TKT NET+FSC_KRW  1858 non-null   float64       
 10  TKT A/R_KRW      1858 non-null   float64       
 11  WTI              1858 non-null   float64       
 12  exchanges        1858 non-null   float64       
 13  kospi            1858 non-null   float64       
 14  rates            1858 non-null   float64

In [8]:
res_data = data[data["Date"].isin(pd.date_range('2016-01-04', '2020-01-31'))]
res_data.reset_index(drop=True, inplace=True)
res_data = res_data[['Date', 'REV OBD']]
scale_cols = ['REV OBD']

In [9]:
res_data.shape

(1489, 2)

In [10]:
# 원본 코드 LSTM-pytorch_optuna_05와 다른 부분

# Loockback_period & forecasting_period
max_prediction_length = 20
lookback_length = 100
training_data_max = len(res_data) - max_prediction_length

# 학습용 데이터
data_p = res_data.iloc[:training_data_max, :]
training_data = scaler.fit_transform(data_p[scale_cols])

In [11]:
training_data_max

1469

In [12]:
data_p.shape

(1469, 2)

In [13]:
data_p

,Date,REV OBD
0,2016-01-04,28000.0
1,2016-01-05,24657.0
2,2016-01-06,26920.0
3,2016-01-07,26624.0
4,2016-01-08,28879.0
...,...,...
1464,2020-01-07,24716.0
1465,2020-01-08,26562.0
1466,2020-01-09,26855.0
1467,2020-01-10,31204.0


#### Metric 생성을 위한 oot sample 정의 (원본 코드 LSTM-pytorch_optuna_05와 다른 부분)


In [14]:
# max_prediction_length 만큼의 데이터는 예측 데이터와 비교를 위해 분리
# Training set에 없는 데이터로 구성
# Input과 output의 pair로 정의
x_for_metric = scaler.fit_transform(res_data[training_data_max -lookback_length : training_data_max][scale_cols])
y_for_metric = scaler.fit_transform(res_data[training_data_max:][scale_cols])

In [15]:
x_for_metric.shape

(100, 1)

In [16]:
y_for_metric.shape

(20, 1)

In [17]:
# LSTM은 1 step 뒤의 값만을 예측하므로, forecasting_period를 1로 두고 진행
x, y = sliding_windows(training_data, lookback_length, 1)

### Model 정의

In [18]:
# 원본 코드 LSTM-pytorch_optuna_05와 다른 부분

class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size,
                            num_layers = num_layers, batch_first = True)
        
        self.fc = nn.Linear(hidden_size  * num_layers, num_classes)
        
    def forward(self, x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size, device = x.device))
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size, device = x.device))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        h_out = h_out.view(-1, self.hidden_size * self.num_layers)
        out = self.fc(h_out)
        return out

In [19]:
def train(log_interval, model, train_dl, val_dl, optimizer, criterion, epoch):

    best_loss = np.inf
    for epoch in range(epoch):
        train_loss = 0.0
        model.train()
        for data, target in train_dl:

            if torch.cuda.is_available():
                data, target = data.cuda(), target.cuda()
                model = model.cuda()

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target) # mean-squared error for regression
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # validation
        valid_loss = 0.0
        model.eval()
        for data, target in val_dl:

            if torch.cuda.is_available():
                data, target = data.cuda(), target.cuda()

            output = model(data)         
            loss = criterion(output, target)
            valid_loss += loss.item()

        if ( epoch % log_interval == 0 ):
            print(f'\n Epoch {epoch} \t Training Loss: {train_loss / len(train_dl)} \t Validation Loss: {valid_loss / len(val_dl)} \n')

        if best_loss > (valid_loss / len(val_dl)):
            print(f'Validation Loss Decreased({best_loss:.6f}--->{(valid_loss / len(val_dl)):.6f}) \t Saving The Model')
            best_loss = (valid_loss / len(val_dl))
            torch.save(model.state_dict(), 'lstm_saved_model.pth')

    return best_loss


def smape(a, f):
    return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)

In [20]:
# 무슨 의미인가?

aa = x_for_metric
tmp = np.append( np.expand_dims(aa[1:, :], 0), np.expand_dims(y_for_metric[2, :], (0,2)), axis=1)
tmp.shape

(1, 100, 1)

In [21]:
def objective(trial):

    cfg = { 
            'batch_size' : trial.suggest_categorical('batch_size',[32, 64, 128, 256, 512]), # [64, 128, 256],
            'num_epochs' : trial.suggest_int('num_epochs', 5, 50, 5),
            'learning_rate' : trial.suggest_loguniform('learning_rate', 1e-3, 1e-1), #trial.suggest_loguniform('learning_rate', 1e-2, 1e-1), # learning rate을 0.01-0.1까지 로그 uniform 분포로 사용
            'hidden_size': trial.suggest_categorical('hidden_size',[32, 64, 128, 256, 512]),
            'num_layers': trial.suggest_int('num_layers', 1, 7, 1),       
        }

    torch.manual_seed(42)

    log_interval = 5
    num_classes = 1 # parameter에서 빼서 상수로 설정

    train_dl, val_dl, input_size = get_data_loader(x, y,  cfg['batch_size'])
    
    model = LSTM(
        num_classes = num_classes, 
        input_size = input_size, 
        hidden_size = cfg['hidden_size'], 
        num_layers = cfg['num_layers']
    )
    
    if torch.cuda.is_available():
        model = model.cuda()
        
    optimizer = optim.Adam(model.parameters(), lr=cfg['learning_rate'])
    criterion = torch.nn.MSELoss()
    best_loss = train(log_interval, model, train_dl, val_dl, optimizer, criterion, cfg['num_epochs'])

    print('best loss for the trial = ', best_loss)
    predict_data = []
    # 여기서 x는 (sample, lookback_length, 1)의 크기를 지님. 따라서, 제일 앞의 시점을 제거하려면, x[:, -1, :]이 되어야 함
    x_pred = np.expand_dims(x_for_metric, 0)  # Inference에 사용할 lookback data를 x_pred로 지정. 앞으로 x_pred를 하나씩 옮겨 가면서 inference를 할 예정

    for j, i in enumerate(range(max_prediction_length)):

        # feed the last forecast back to the model as an input
        x_pred = np.append( x_pred[:, 1:, :], np.expand_dims(y_for_metric[j, :], (0,2)), axis=1)
        xt_pred = torch.Tensor(x_pred)

        if torch.cuda.is_available():
            xt_pred = xt_pred.cuda()
        # generate the next forecast
        yt_pred = model(xt_pred)
        # tensor to array
        # x_pred = xt_pred.cpu().detach().numpy()
        y_pred = yt_pred.cpu().detach().numpy()

        # save the forecast
        predict_data.append(y_pred)

    # transform the forecasts back to the original scale
    predict_data = np.array(predict_data).reshape(-1, 1)
    SMAPE = smape(y_for_metric, predict_data)
    
    print(f' \nSMAPE : {SMAPE}')


    return SMAPE

In [22]:
sampler = optuna.samplers.TPESampler()
#   sampler = optuna.samplers.SkoptSampler()

# model.load_state_dict(torch.load('lstm_saved_model.pth'))
    
study = optuna.create_study(sampler=sampler, direction='minimize')
study.optimize(objective, n_trials= 7)
joblib.dump(study, '/content/drive/MyDrive/교육/AI실무인증과정/LSTM/LSTM_pytorch_optuna_06.pkl')

[I 2022-06-22 11:33:58,825] A new study created in memory with name: no-name-29dc2960-7f0a-41a0-9080-9d3d780c8ce1



 Epoch 0 	 Training Loss: 0.05748009060819944 	 Validation Loss: 0.02064674347639084 

Validation Loss Decreased(inf--->0.020647) 	 Saving The Model
Validation Loss Decreased(0.020647--->0.018611) 	 Saving The Model
Validation Loss Decreased(0.018611--->0.018445) 	 Saving The Model
Validation Loss Decreased(0.018445--->0.016424) 	 Saving The Model

 Epoch 5 	 Training Loss: 0.023175305169489648 	 Validation Loss: 0.016599976768096287 


 Epoch 10 	 Training Loss: 0.023078636783692572 	 Validation Loss: 0.016761823867758114 


 Epoch 15 	 Training Loss: 0.02305650297138426 	 Validation Loss: 0.016741549596190453 


 Epoch 20 	 Training Loss: 0.02300599817600515 	 Validation Loss: 0.01668694739540418 


 Epoch 25 	 Training Loss: 0.02295179044206937 	 Validation Loss: 0.01663207480063041 


 Epoch 30 	 Training Loss: 0.02290134442349275 	 Validation Loss: 0.01658148504793644 


 Epoch 35 	 Training Loss: 0.02285603516631656 	 Validation Loss: 0.016535097733139992 


 Epoch 40 	 Training

[I 2022-06-22 11:34:19,663] Trial 0 finished with value: 65.2645984124806 and parameters: {'batch_size': 128, 'num_epochs': 45, 'learning_rate': 0.0038091057345700946, 'hidden_size': 128, 'num_layers': 2}. Best is trial 0 with value: 65.2645984124806.


best loss for the trial =  0.01642377519359191
 
SMAPE : 65.2645984124806

 Epoch 0 	 Training Loss: 573.9674616754055 	 Validation Loss: 719.6298828125 

Validation Loss Decreased(inf--->719.629883) 	 Saving The Model
Validation Loss Decreased(719.629883--->489.581757) 	 Saving The Model
Validation Loss Decreased(489.581757--->211.792694) 	 Saving The Model
Validation Loss Decreased(211.792694--->9.751367) 	 Saving The Model

 Epoch 5 	 Training Loss: 59.9263718922933 	 Validation Loss: 117.9341812133789 

Validation Loss Decreased(9.751367--->5.249639) 	 Saving The Model

 Epoch 10 	 Training Loss: 21.697325229644775 	 Validation Loss: 10.497438430786133 

Validation Loss Decreased(5.249639--->1.630319) 	 Saving The Model

 Epoch 15 	 Training Loss: 4.530174712340037 	 Validation Loss: 3.284949541091919 

Validation Loss Decreased(1.630319--->0.438770) 	 Saving The Model
Validation Loss Decreased(0.438770--->0.433106) 	 Saving The Model

 Epoch 20 	 Training Loss: 0.8404969424009323 

[I 2022-06-22 11:35:10,960] Trial 1 finished with value: 71.48786428210035 and parameters: {'batch_size': 512, 'num_epochs': 50, 'learning_rate': 0.06609874673957551, 'hidden_size': 512, 'num_layers': 5}. Best is trial 0 with value: 65.2645984124806.


...

[I 2022-06-22 11:35:43,250] Trial 5 finished with value: 64.25869960736114 and parameters: {'batch_size': 32, 'num_epochs': 25, 'learning_rate': 0.05217185390739443, 'hidden_size': 32, 'num_layers': 4}. Best is trial 5 with value: 64.25869960736114.


Validation Loss Decreased(0.016878--->0.016844) 	 Saving The Model
best loss for the trial =  0.016843916848301888
 
SMAPE : 64.25869960736114

 Epoch 0 	 Training Loss: 4.182005787889163 	 Validation Loss: 0.22810227870941163 

Validation Loss Decreased(inf--->0.228102) 	 Saving The Model
Validation Loss Decreased(0.228102--->0.079386) 	 Saving The Model
Validation Loss Decreased(0.079386--->0.054940) 	 Saving The Model
Validation Loss Decreased(0.054940--->0.022207) 	 Saving The Model

 Epoch 5 	 Training Loss: 0.02885263951288329 	 Validation Loss: 0.01902327537536621 

Validation Loss Decreased(0.022207--->0.019023) 	 Saving The Model
Validation Loss Decreased(0.019023--->0.018665) 	 Saving The Model

 Epoch 10 	 Training Loss: 0.03049100014484591 	 Validation Loss: 0.026043500006198882 


 Epoch 15 	 Training Loss: 0.04376911206377877 	 Validation Loss: 0.02007005363702774 


 Epoch 20 	 Training Loss: 0.045665818059609994 	 Validation Loss: 0.07604395225644112 


 Epoch 25 	 Trai

[I 2022-06-22 11:36:40,391] Trial 6 finished with value: 80.90670728743356 and parameters: {'batch_size': 64, 'num_epochs': 35, 'learning_rate': 0.005986233864334597, 'hidden_size': 512, 'num_layers': 5}. Best is trial 5 with value: 64.25869960736114.


best loss for the trial =  0.01866472288966179
 
SMAPE : 80.90670728743356


['/content/drive/MyDrive/교육/AI실무인증과정/LSTM/LSTM_pytorch_optuna_06.pkl']

In [23]:
study = joblib.load('/content/drive/MyDrive/교육/AI실무인증과정/LSTM/LSTM_pytorch_optuna_06.pkl') 
df = study.trials_dataframe() 
df.sort_values('value')

,number,value,datetime_start,datetime_complete,duration,params_batch_size,params_hidden_size,params_learning_rate,params_num_epochs,params_num_layers,state
5,5,64.258700,2022-06-22 11:35:30.269737,2022-06-22 11:35:43.250624,0 days 00:00:12.980887,32,32,0.052172,25,4,COMPLETE
0,0,65.264598,2022-06-22 11:33:58.832678,2022-06-22 11:34:19.662926,0 days 00:00:20.830248,128,128,0.003809,45,2,COMPLETE
3,3,66.497066,2022-06-22 11:35:16.333515,2022-06-22 11:35:18.986343,0 days 00:00:02.652828,512,32,0.025897,50,3,COMPLETE
2,2,67.335145,2022-06-22 11:35:10.961908,2022-06-22 11:35:16.331743,0 days 00:00:05.369835,256,256,0.006021,45,1,COMPLETE
1,1,71.487864,2022-06-22 11:34:19.666222,2022-06-22 11:35:10.960080,0 days 00:00:51.293858,512,512,0.066099,50,5,COMPLETE
4,4,80.762639,2022-06-22 11:35:18.988171,2022-06-22 11:35:30.267761,0 days 00:00:11.279590,64,32,0.012539,25,6,COMPLETE
6,6,80.906707,2022-06-22 11:35:43.254234,2022-06-22 11:36:40.391254,0 days 00:00:57.137020,64,512,0.005986,35,5,COMPLETE


In [24]:
study.best_params

{'batch_size': 32,
 'hidden_size': 32,
 'learning_rate': 0.05217185390739443,
 'num_epochs': 25,
 'num_layers': 4}

In [25]:
optuna.visualization.plot_param_importances(study)

In [26]:
optuna.visualization.plot_optimization_history(study)

In [27]:
# 실제 예측하려는 target 구간의 max_encoder_length : 100일 
test_data = res_data[res_data["Date"].isin(pd.date_range('2019-10-24', '2020-01-31'))]
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 1389 to 1488
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     100 non-null    datetime64[ns]
 1   REV OBD  100 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 2.3 KB


In [28]:
test_data.shape

(100, 2)

In [29]:
torch.manual_seed(42)

log_interval = 5
num_classes = 1 # parameter에서 빼서 상수로 설정

train_dl, val_dl, input_size = get_data_loader(x, y,  study.best_params['batch_size'])

model = LSTM(
    num_classes = num_classes, 
    input_size = input_size, 
    hidden_size = study.best_params['hidden_size'], 
    num_layers = study.best_params['num_layers']
)

if torch.cuda.is_available():
    model = model.cuda()
    
optimizer = optim.Adam(model.parameters(), lr=study.best_params['learning_rate'])
criterion = torch.nn.MSELoss()
best_loss = train(log_interval, model, train_dl, val_dl, optimizer, criterion, study.best_params['num_epochs'])

print('best loss for the trial = ', best_loss)
predict_data = []
# 여기서 x는 (sample, lookback_length, 1)의 크기를 지님. 따라서, 제일 앞의 시점을 제거하려면, x[:, -1, :]이 되어야 함
x_pred = np.expand_dims(x_for_metric, 0)  # Inference에 사용할 lookback data를 x_pred로 지정. 앞으로 x_pred를 하나씩 옮겨 가면서 inference를 할 예정

for j, i in enumerate(range(max_prediction_length)):

    # feed the last forecast back to the model as an input
    x_pred = np.append( x_pred[:, 1:, :], np.expand_dims(y_for_metric[j, :], (0,2)), axis=1)
    xt_pred = torch.Tensor(x_pred)

    if torch.cuda.is_available():
        xt_pred = xt_pred.cuda()
    # generate the next forecast
    yt_pred = model(xt_pred)
    # tensor to array
    # x_pred = xt_pred.cpu().detach().numpy()
    y_pred = yt_pred.cpu().detach().numpy()

    # save the forecast
    predict_data.append(y_pred)

# transform the forecasts back to the original scale
predict_data = np.array(predict_data).reshape(-1, 1)
SMAPE = smape(y_for_metric, predict_data)

print(f' \nSMAPE : {SMAPE}')


 Epoch 0 	 Training Loss: 0.11322279231888907 	 Validation Loss: 0.01789849054896169 

Validation Loss Decreased(inf--->0.017898) 	 Saving The Model
Validation Loss Decreased(0.017898--->0.017869) 	 Saving The Model
Validation Loss Decreased(0.017869--->0.017253) 	 Saving The Model
Validation Loss Decreased(0.017253--->0.017139) 	 Saving The Model

 Epoch 5 	 Training Loss: 0.023498190247586797 	 Validation Loss: 0.0171440981535448 


 Epoch 10 	 Training Loss: 0.023569034731813838 	 Validation Loss: 0.017685135309067037 

Validation Loss Decreased(0.017139--->0.017096) 	 Saving The Model
Validation Loss Decreased(0.017096--->0.016878) 	 Saving The Model

 Epoch 15 	 Training Loss: 0.022819578221866064 	 Validation Loss: 0.017294477878345385 


 Epoch 20 	 Training Loss: 0.022853820105748518 	 Validation Loss: 0.01706583332270384 

Validation Loss Decreased(0.016878--->0.016844) 	 Saving The Model
best loss for the trial =  0.016843916848301888
 
SMAPE : 64.25869960736114


In [30]:
# actual_data는 max_prediction_length에 해당하는 20일의 기간 : 실제 예측하려는 기간 (2020년 2월1일 ~ 2020년 2월 20일)
actual_data = data[data["Date"].isin(pd.date_range('2020-02-01', '2020-02-20'))]
actual_data['time_index'] = np.arange(len(actual_data))
actual_data['market'] = 'OBD'
actual_data.head()

,Date,Account DOW,REV OBD,OBD NET+FSC_KRW,OBD A/R_KRW,REV CPN,CPN NET+FSC_KRW,CPN A/R_KRW,REV TKT,TKT NET+FSC_KRW,TKT A/R_KRW,WTI,exchanges,kospi,rates,stock_a,stock_k,stock_kkj,time_index,market
1492,2020-02-01,SAT,23210.0,1.172142e+10,505015.9678,6412.0,2.781090e+09,433732.0616,3888.0,3.517716e+09,9.047624e+05,51.56,1194.5,2119.01,1.30,21763.0,15335.0,43860.0,0,OBD
1493,2020-02-02,SUN,20824.0,9.549017e+09,458558.2667,6049.0,2.553490e+09,422134.2552,3675.0,2.836992e+09,7.719705e+05,51.56,1194.5,2119.01,1.30,21763.0,15335.0,43860.0,1,OBD
1494,2020-02-03,MON,19331.0,8.695239e+09,449808.0166,17749.0,1.013166e+10,570829.9001,10027.0,1.117746e+10,1.114736e+06,50.11,1194.0,2118.88,1.29,21883.0,15659.0,44739.0,2,OBD
1495,2020-02-04,TUE,17277.0,7.375559e+09,426900.4457,16530.0,9.081702e+09,549407.2659,9351.0,1.003142e+10,1.072765e+06,49.61,1188.5,2157.90,1.33,22790.0,16697.0,47035.0,3,OBD
1496,2020-02-05,WED,17211.0,7.884009e+09,458079.6780,18526.0,1.005432e+10,542713.8500,10438.0,1.124609e+10,1.077418e+06,50.75,1184.5,2165.63,1.31,22909.0,16859.0,46937.0,4,OBD


In [31]:
# 실제 예측하려는 target prediction 구간 : 20일
actuals = actual_data['REV OBD']
actuals

1492    23210.0
1493    20824.0
1494    19331.0
1495    17277.0
1496    17211.0
1497    16299.0
1498    18470.0
1499    17570.0
1500    17037.0
1501    16587.0
1502    13758.0
1503    14898.0
1504    14091.0
1505    16195.0
1506    16074.0
1507    15017.0
1508    14490.0
1509    11933.0
1510    13100.0
1511    12061.0
Name: REV OBD, dtype: float64

In [32]:
actuals = torch.tensor(actual_data['REV OBD'].values)
print(actuals)
print(actuals.shape)

tensor([23210., 20824., 19331., 17277., 17211., 16299., 18470., 17570., 17037.,
        16587., 13758., 14898., 14091., 16195., 16074., 15017., 14490., 11933.,
        13100., 12061.], dtype=torch.float64)
torch.Size([20])


In [33]:
baseline_predictions =  torch.tensor([data.loc[data.Date == '2020-01-31']['REV OBD'].values[0]] * len(actuals)).to(torch.float32)

mae_with_baseline = (actuals - baseline_predictions).abs().mean().item()
smape_with_baseline = (2 * (baseline_predictions - actuals).abs() / (baseline_predictions.abs() + actuals.abs() + 1e-8)).mean()

print(f'mae_with_baseline : {mae_with_baseline}')
print(f'smape_with_baseline : {smape_with_baseline}')

mae_with_baseline : 7632.35
smape_with_baseline : 0.39083401983455535


In [38]:
torch.tensor([data.loc[data.Date == '2020-01-31']['REV OBD'].values[0]] * len(actuals)).to(torch.float32)

tensor([23904., 23904., 23904., 23904., 23904., 23904., 23904., 23904., 23904.,
        23904., 23904., 23904., 23904., 23904., 23904., 23904., 23904., 23904.,
        23904., 23904.])

In [34]:
predicts = scaler.inverse_transform(predict_data)
print(predicts.shape)
print(predicts)
print(type(predicts))

(20, 1)
[[25458.75 ]
 [25387.09 ]
 [25091.88 ]
 [25435.121]
 [25151.662]
 [25639.234]
 [25292.557]
 [25087.676]
 [24852.074]
 [24863.977]
 [25256.951]
 [25193.934]
 [24965.936]
 [24552.059]
 [25093.479]
 [25416.496]
 [25019.54 ]
 [24737.062]
 [24675.943]
 [24816.951]]
<class 'numpy.ndarray'>


In [35]:
predicts = torch.tensor(predicts)
predicts.shape

torch.Size([20, 1])

In [36]:
mae_with_test = (actuals - predicts).abs().mean().item()
smape_with_test = smape(actuals.numpy(), predicts.numpy())

print(f'mae_with_test : {mae_with_test}')
print(f'smape_with_test : {smape_with_test}')

mae_with_test : 8827.7685546875
smape_with_test : 874.3154316016535


In [37]:
y_train.shape

np.expand_dims(pred, axis=1).shape

predicted_for_plot = sc.inverse_transform(np.concatenate((y_train, np.expand_dims(pred, axis=1))))
actual_for_plot = sc.inverse_transform(np.concatenate((y_train, y_test)))

predicted_for_plot.shape

actual_for_plot.shape

plt.plot(predicted_for_plot[300:])
plt.plot(actual_for_plot[300:])
plt.suptitle('Timeseries Prediction')
plt.axvline(x=y_train.shape[0]-300, c='r', linestyle='--')
plt.show()

NameError: ignored